In [1]:
# This script is meant to match the functionality of nc2RAS.m Matlab script
# Converts an input netCDF file to be used  for the 500 different equiprobable rainfall fields derived from the IPET predicted rainfall for selected historic rainfall events

# Import List of selected historic rainfall events and their respective startTimes
import glob, os
import pandas as pd
from datetime import datetime
import xarray as xr
stormListFileName = "LWI_Phase2Storms_EBTRK_Times.csv"
stormList = pd.read_csv(stormListFileName)
stormList

,Name,StormID,StartTime,EndTime
0,Tropical Storm Bertha (2002),Bertha_2002,8/4/02 18:00,8/9/02 12:00
1,Hurricane Isidore (2002),Isidore_2002,9/14/02 18:00,9/27/02 18:00
2,Hurricane Lili (2002),Lili_2002,9/21/02 8:00,10/4/02 12:00
3,Tropical Storm Bill (2003),Bill_2003,6/28/03 6:00,7/30/03 0:00
4,Tropical Storm Matthew (2004),Matthew_2004,10/8/04 12:00,10/11/04 6:00
5,Hurricane Cindy (2005),Cindy_2005,7/3/05 18:00,7/11/05 6:00
6,Hurricane Rita (2005),Rita_2005,9/18/05 0:00,9/26/05 6:00
7,Hurricane Gustav (2008),Gustav_2008,8/25/08 0:00,9/5/08 12:00
8,Tropical Storm Bonnie (2010),Bonnie_2010,7/22/10 6:00,7/25/10 18:00
9,Tropical Storm Lee (2011),Lee_2011,9/2/11 0:00,9/6/11 18:00


In [2]:
#Check folder for input files
os.chdir("./input")
filenameList = []
for file in glob.glob("*.nc"):
    filenameList.append(file)
filenameList
os.chdir("..")

In [3]:
#Loop through input files
for file in filenameList:
    outFilename = file.split(".")[0] + "-hec.nc"
    stormID = file.split(".")[0].split("-")[2]
    ensembleID = file.split(".")[0].split("-")[3]
    # Look up startTime from stormList by stormID
    try:
        startTime_str = stormList[stormList['StormID'].str.contains(stormID, na=False, case=False)]['StartTime'].values[0]
    except:
        print (f'{stormID} not found in {stormListFileName}')
    startTime_dt = datetime.strptime(startTime_str, '%m/%d/%y %H:%M')
    # .time.attrs['units'] = 'minutes since 2021-08-24 00:00:00'
    startTime_str_formatted = datetime.strftime(startTime_dt, '%Y-%m-%d %H:%M:00')

In [4]:
# from here and cells below - Add back to input file loop later

# open input nc, edit to make HEC compliant, save as output nc, run through vortex jython api to create dss file.
ds = xr.open_dataset("./input/"+file)
ds

<xarray.Dataset>
Dimensions:   (timestep: 846, lat: 122, lon: 144)
Coordinates:
  * timestep  (timestep) float32 0.0 20.0 40.0 ... 1.686e+04 1.688e+04 1.69e+04
  * lat       (lat) float32 27.95 28.0 28.05 28.1 ... 33.85 33.9 33.95 34.0
  * lon       (lon) float32 -95.0 -94.95 -94.9 -94.85 ... -87.95 -87.9 -87.85
Data variables:
    tcr       (timestep, lat, lon) float32 ...
Attributes:
    unit:     mm/20min

In [5]:
varNames = {'lon': 'x', 'lat': 'y', 'timestep': 'time', 'tcr': 'rain'}
ds = ds.rename(varNames)
ds

<xarray.Dataset>
Dimensions:  (time: 846, y: 122, x: 144)
Coordinates:
  * time     (time) float32 0.0 20.0 40.0 60.0 ... 1.686e+04 1.688e+04 1.69e+04
  * y        (y) float32 27.95 28.0 28.05 28.1 28.15 ... 33.85 33.9 33.95 34.0
  * x        (x) float32 -95.0 -94.95 -94.9 -94.85 ... -88.0 -87.95 -87.9 -87.85
Data variables:
    rain     (time, y, x) float32 ...
Attributes:
    unit:     mm/20min

In [6]:
ds.time.attrs['units'] = f'minutes since {startTime_str_formatted}'
ds.time.attrs['standard_name'] = 'time'
ds.time.attrs['long_name'] = 'time'
ds.time.attrs['axis'] = 'T'

In [7]:
ds.x.attrs['units'] = 'degrees_east'
ds.y.attrs['units'] = 'degrees_north'
ds.x.attrs['long_name'] = 'Longitude'
ds.y.attrs['long_name'] = 'Latitude'
ds.x.attrs['axis'] = 'X'
ds.y.attrs['axis'] = 'Y'
ds

<xarray.Dataset>
Dimensions:  (time: 846, y: 122, x: 144)
Coordinates:
  * time     (time) float32 0.0 20.0 40.0 60.0 ... 1.686e+04 1.688e+04 1.69e+04
  * y        (y) float32 27.95 28.0 28.05 28.1 28.15 ... 33.85 33.9 33.95 34.0
  * x        (x) float32 -95.0 -94.95 -94.9 -94.85 ... -88.0 -87.95 -87.9 -87.85
Data variables:
    rain     (time, y, x) float32 ...
Attributes:
    unit:     mm/20min

In [8]:
ds = ds.assign({
    'crs' : (
        (),
        0
    )
})



In [10]:
ds.crs.attrs = {
    'long_name': 'coordinate reference system',
    'epsg_code': 'EPSG:4326',
    'crs_wkt' : 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]',
    'semi_major_axis' : 6378137.0,
    'semi_minor_axis' : 6356752.314245179,
    'inverse_flattening' : 298.257223563,
    'reference_ellipsoid_name' : 'WGS 84',
    'longitude_of_prime_meridian' : 0.0,
    'prime_meridian_name' : 'Greenwich',
    'geographic_crs_name' : 'WGS 84',
    'grid_mapping_name' : 'latitude_longitude',
    'spatial_ref' : 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]',
}
ds

<xarray.Dataset>
Dimensions:  (time: 846, y: 122, x: 144)
Coordinates:
  * time     (time) float32 0.0 20.0 40.0 60.0 ... 1.686e+04 1.688e+04 1.69e+04
  * y        (y) float32 27.95 28.0 28.05 28.1 28.15 ... 33.85 33.9 33.95 34.0
  * x        (x) float32 -95.0 -94.95 -94.9 -94.85 ... -88.0 -87.95 -87.9 -87.85
Data variables:
    rain     (time, y, x) float32 ...
    crs      int32 0
Attributes:
    unit:     mm/20min

In [ ]:
# import rioxarray as rxr
# ds.rio.write_crs('epsg:4326', inplace=True)
# ds = ds.rename({'spatial_ref':'crs'})
# ds.crs.attrs['long_name'] = 'coordinate reference system'
# ds.crs.attrs['epsg_code'] = 'EPSG:4326'
# ds


In [11]:
ds.rain.attrs['long_name'] = 'Total rainfall accumulation over 20 minutes'
ds.rain.attrs['units'] = 'mm'
# ncwriteatt(ncfileout,'UD','grid_mapping','crs')
ds.rain.attrs['grid_mapping'] = 'crs'
ds.rain.attrs['_FillValue'] = -999

In [12]:
ds.attrs = {
    'Conventions':'CF-1.6,UGRID-0.9',
    'title': f'{stormID}',
    'ensembleID': f'{ensembleID}',
    'institution': 'The Water Institute',
    'source': f'LWI Storm Generator {file}',
    'Metadata_Conventions': 'Unidata Dataset Discovery v1.0',
    'summary': 'LWI_Phase2_Synthetic_Rainfall. 500 different equiprobable rainfall fields derived from the IPET predicted rainfall for selected historic rainfall events',
    'date_created': f"{datetime.today().strftime('%Y-%m-%d')}",
    }

In [13]:
ds

<xarray.Dataset>
Dimensions:  (time: 846, y: 122, x: 144)
Coordinates:
  * time     (time) float32 0.0 20.0 40.0 60.0 ... 1.686e+04 1.688e+04 1.69e+04
  * y        (y) float32 27.95 28.0 28.05 28.1 28.15 ... 33.85 33.9 33.95 34.0
  * x        (x) float32 -95.0 -94.95 -94.9 -94.85 ... -88.0 -87.95 -87.9 -87.85
Data variables:
    rain     (time, y, x) float32 ...
    crs      int32 0
Attributes:
    Conventions:           CF-1.6,UGRID-0.9
    title:                 BERTHA_2002
    ensembleID:            ENS1
    institution:           The Water Institute
    source:                LWI Storm Generator tcr-20min-BERTHA_2002-ENS1-fak...
    Metadata_Conventions:  Unidata Dataset Discovery v1.0
    summary:               LWI_Phase2_Synthetic_Rainfall. 500 different equip...
    date_created:          2022-07-20

In [14]:
ds.to_netcdf(path="./output/"+outFilename, format="NETCDF4", engine="netcdf4")

In [15]:
!jupyter nbconvert --to script [nc2RAS].ipynb

_cffi_ext.c
C:\Tools\Anaconda3\lib\site-packages\zmq\backend\cffi\__pycache__\_cffi_ext.c(268): fatal error C1083: Cannot open include file: 'zmq.h': No such file or directory


Traceback (most recent call last):
  File "C:\Tools\Anaconda3\Scripts\jupyter-nbconvert-script.py", line 6, in <module>
    from nbconvert.nbconvertapp import main
  File "C:\Tools\Anaconda3\lib\site-packages\nbconvert\__init__.py", line 4, in <module>
    from .exporters import *
  File "C:\Tools\Anaconda3\lib\site-packages\nbconvert\exporters\__init__.py", line 4, in <module>
    from .slides import SlidesExporter
  File "C:\Tools\Anaconda3\lib\site-packages\nbconvert\exporters\slides.py", line 12, in <module>
    from ..preprocessors.base import Preprocessor
  File "C:\Tools\Anaconda3\lib\site-packages\nbconvert\preprocessors\__init__.py", line 10, in <module>
    from .execute import ExecutePreprocessor
  File "C:\Tools\Anaconda3\lib\site-packages\nbconvert\preprocessors\execute.py", line 8, in <module>
    from nbclient import NotebookClient, execute as _execute
  File "C:\Tools\Anaconda3\lib\site-packages\nbclient\__init__.py", line 3, in <module>
    from .client import Notebook